In [1]:
import pandas as pd

In [7]:
# train = pd.read_csv('D:\system_default\desktop\仓库\MonTv-RS-solo28\MongoTv\data_v2\main_vv_seq_train.csv')
# train_a = pd.read_csv('D:\system_default\desktop\仓库\MonTv-RS-solo28\MongoTv\data_v2\main_vv_seq_train_A.csv')

vid_info = pd.read_csv('./data_v2/vid_info.csv')
candidat = pd.read_csv('data_v2\candidate_items_A.csv')


In [6]:
candi.shape

(149626, 12)

In [4]:
train.head(2)
print(train.shape)
train.head(10)

(5596413, 8)


,did,vid,vts,hb,seq_no,cpn,fpn,time_gap
0,5ebf5b53f2fdec1eadba29c3b2155e8e,83cf1332aa7315c755cc8bdd356513af,111.0,84.0,5.0,1,130,87.0
1,5ebf5b53f2fdec1eadba29c3b2155e8e,18d443cacff1173b21ea4c28c8bdf3b0,114.0,85.0,4.0,1,130,115.0
2,5ebf5b53f2fdec1eadba29c3b2155e8e,27d332d1ad1a58e205b797be64d30e13,137.0,80.0,3.0,1,130,75.0
3,5ebf5b53f2fdec1eadba29c3b2155e8e,2ac8e7855bb39cd1dace86c7cd5e72b7,91.0,74.0,2.0,1,130,90.0
4,5ebf5b53f2fdec1eadba29c3b2155e8e,941e1b0a4e8d5ae20ab328f21e8bfcc0,118.0,86.0,1.0,1,130,NaN
5,76025b9e4c39cbcf416b0815003eb6bc,98c0124136a942e2a41aee003f4f1298,317.0,392.0,76.0,1,130,7.0
6,76025b9e4c39cbcf416b0815003eb6bc,db978f94b983a6ed3021e1e927ee09ef,8.0,89.0,75.0,1,68,12236.0
7,76025b9e4c39cbcf416b0815003eb6bc,4a87208053004ae71e82c4af599de242,444.0,439.0,74.0,1,130,6126.0
8,76025b9e4c39cbcf416b0815003eb6bc,0cc56b6914c87d906b23e6d90cd4f65e,6116.0,6146.0,73.0,1,68,68555.0
9,76025b9e4c39cbcf416b0815003eb6bc,f220c547e9e127dbf849edb0adf3df04,20.0,2.0,72.0,1,68,49.0


In [3]:
vid_info = pd.read_csv('../input/vid_info.csv')

In [4]:
candidate_items = pd.read_csv('../input/candidate_items_A.csv')

In [6]:
train = train.merge(vid_info[['vid','cid']],on='vid',how='left')

In [7]:
train = train[['did','vid','cid']]

In [8]:
train['did_count'] = train['did'].map(train.did.value_counts())

In [9]:
train['rank'] =  train.groupby('did').cumcount()+1

In [10]:
train['vid_next'] = train.groupby(['did']).vid.shift(1) # 这个是很重要的

In [11]:
tmp = train.groupby(['vid','vid_next']).size().sort_values(ascending=False).reset_index()

In [12]:
tmp = tmp[tmp.vid_next.isin(candidate_items.vid.unique())]

In [13]:
tmp = tmp.groupby('vid').head(6)

In [14]:
base =  train[train['rank']==1][['did','vid']]

In [15]:
base = base.merge(tmp[['vid','vid_next',0]],on='vid')

In [16]:
hot_list = train[train['rank']==1].vid.value_counts().head(6).index

In [17]:
base5 = train.did.drop_duplicates().to_frame()

In [18]:
base5['key'] =1

In [19]:
top_hot_list = pd.DataFrame(hot_list, columns=['vid_next'])

In [20]:
top_hot_list['key'] = 1

In [21]:
base5 = base5.merge(top_hot_list,on=['key'],how='inner')

In [22]:
base_all = pd.concat([base,base5])

In [23]:
base_all = base_all[base_all.vid_next.isin(candidate_items.vid.unique())]

In [24]:
base_all = base_all[base_all.vid_next!=base_all.vid]

In [25]:
base_all = base_all.drop_duplicates(['did','vid_next'],keep='first')

In [26]:
base_all = base_all.groupby('did').head(6)

In [27]:
base_all['rank'] = base_all.groupby('did').cumcount()+1

In [28]:
sub = base_all[['did', 'vid_next',  'rank']]

In [29]:
sub.columns = ['did', 'vid', 'rank']

In [30]:
sub.to_csv('res.csv',index=None)

In [31]:
sub

,did,vid,rank
0,5ebf5b53f2fdec1eadba29c3b2155e8e,5bc78a50602b520bb3f6c87e3c542f1c,1
1,5ebf5b53f2fdec1eadba29c3b2155e8e,5586977e241d184ecd47576a12687576,2
2,5ebf5b53f2fdec1eadba29c3b2155e8e,0cc56b6914c87d906b23e6d90cd4f65e,3
3,5ebf5b53f2fdec1eadba29c3b2155e8e,db978f94b983a6ed3021e1e927ee09ef,4
4,5ebf5b53f2fdec1eadba29c3b2155e8e,98c0124136a942e2a41aee003f4f1298,5
...,...,...,...
1024965,a40790db7c6e0a3ff583c225c9638d7b,64ece3758213c9b08da38832b3bc49f3,4
1024966,a40790db7c6e0a3ff583c225c9638d7b,239fa8776d81792a7477053184e7d6af,5
1024967,a40790db7c6e0a3ff583c225c9638d7b,dd4ad1276e13603b6a15704295ba6ec8,6
1025004,4707b1fa31569c6950c0a46dc904cf9e,0cc56b6914c87d906b23e6d90cd4f65e,5


In [7]:
input_path = '../uspppm_final_bad/'
train_df = pd.read_csv(f"{input_path}train.csv")
titles = pd.read_csv('../uspppm_final_bad/titles.csv')
train_df = train_df.merge(titles, left_on='context', right_on='code')

In [8]:
train_df.head(10)

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
5,067203128142739c,abatement,greenhouse gases,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
6,061d17f04be2d1cf,abatement,increased rate,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
7,e1f44e48399a2027,abatement,measurement level,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
8,0a425937a3e86d10,abatement,minimising sounds,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
9,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN
